In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib.pyplot as plt

# functions

In [2]:
def extractz500_several_files(filestemp):
    listxarrays = []
    for file in filestemp:
        dstemp = xr.open_dataset(file)
        dstemp = dstemp.sel(time=slice('1994-01-01', '2100-12-31'))
        dstemp = dstemp.TREFHTMX.where((dstemp.lat>=10),drop=True)
        # Transpose the data to match the desired dimension order
        dstemp_transposed = dstemp.transpose('time', 'lat', 'lon')
        lat0=20; lat1=80; lon0=180; lon1=330
        dstemp_transposed = dstemp_transposed.where((dstemp_transposed.lat>=lat0)&(dstemp_transposed.lat<=lat1)&\
                               (dstemp_transposed.lon>=lon0)&(dstemp_transposed.lon<=lon1),drop=True)
        
        # Recreate the DataArray with the coordinates in the desired order
        dstemp_reordered = xr.DataArray(
            dstemp_transposed.values,
            dims=['time', 'lat', 'lon'],
            coords={
                'time': dstemp_transposed.coords['time'],
                'lat': dstemp_transposed.coords['lat'],
                'lon': dstemp_transposed.coords['lon']
            },
            attrs=dstemp.attrs,
            name=dstemp.name
        )
        listxarrays.append(dstemp_reordered)
    xarrayfull = xr.concat(listxarrays, dim='time')
    xarrayfull = xarrayfull.drop_duplicates('time',keep='first')
    cftime_index = xr.coding.cftimeindex.CFTimeIndex(xarrayfull['time'].values)
    datetime_index = cftime_index.to_datetimeindex()
    xarrayfull.coords['time'] = datetime_index
    return xarrayfull.to_dataset()

In [3]:
def compute_anomalies(dataset,namevar):
    climatology = (
        dataset[namevar].groupby('time.dayofyear')
        .mean(dim='time')
        .rename({'dayofyear': 'day'})
    )
    climatology_padded = xr.concat([climatology[-30:], climatology, climatology[:30]], dim='day')
    climatology_smoothed = climatology_padded.rolling(day=60, center=True).mean()
    climatology_smoothed = climatology_smoothed.isel(day=slice(30, -30))
    
    climatology_expanded = xr.DataArray(
        climatology_smoothed.sel(day=dataset[namevar]['time.dayofyear']),
        dims=["time", "lat", "lon"],
        coords={"time": dataset[namevar]['time'], "lat": dataset[namevar]['lat'], "lon": dataset[namevar]['lon']}
    )
    
    anomalies = dataset[namevar] - climatology_expanded
    
    climatology_std = (
        dataset[namevar].groupby('time.dayofyear')
        .std(dim='time')
        .rename({'dayofyear': 'day'})
    )
    
    climatology_std_padded = xr.concat([climatology_std[-30:], climatology_std, climatology_std[:30]], dim='day')
    climatology_std_smoothed = climatology_std_padded.rolling(day=60, center=True).mean()
    climatology_std_smoothed = climatology_std_smoothed.isel(day=slice(30, -30))
    climatology_std_expanded = xr.DataArray(
        climatology_std_smoothed.sel(day=dataset[namevar]['time.dayofyear']),
        dims=["time", "lat", "lon"],
        coords={"time": dataset[namevar]['time'], "lat": dataset[namevar]['lat'], "lon": dataset[namevar]['lon']}
    )

    anomalies_std = anomalies/climatology_std_expanded
    return anomalies_std.to_dataset()

# compute

In [4]:
path_temp_files = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/day_1/TREFHTMX/'

In [5]:
filenames = np.sort(glob.glob(f'{path_temp_files}*'))

type_aerosols = []
for i in range(len(filenames)):
    if 'cmip6' in filenames[i].split('.')[2]:
        type_aerosols.append('cmip6')
    elif 'smbb' in filenames[i].split('.')[2]:
        type_aerosols.append('smbb')

In [6]:
path_outputs_anoms = '/glade/derecho/scratch/jhayron/Data4WRsClimateChange/LENS2_mon_maxtemp_anoms/'

In [7]:
names_experiments_all_0 = np.array(['.'.join(filenames[i].split('.')[4:-5]) for i in range(len(filenames))])
names_experiments_all = []
for i in range(len(filenames)):
    names_experiments_all.append(f'{names_experiments_all_0[i]}_{type_aerosols[i]}')
names_experiments_all = np.array(names_experiments_all)
unique_names_experiments = np.unique(names_experiments_all)

In [8]:
# id_experiment = 0
# name_experiment = unique_names_experiments[id_experiment]
# print(name_experiment)
# where_files = np.where(names_experiments_all==name_experiment)[0]
# files_temp = filenames[where_files]

# dataset_raw = extractz500_several_files(files_temp)
# land_mask_file='/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/LANDFRAC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.LANDFRAC.185001-185912.nc'
# namevar='TREFHTMX'
# lsm = xr.open_dataset(land_mask_file).LANDFRAC[0]
# lsm = lsm.sel(lat=dataset_raw.lat,lon=dataset_raw.lon)
# lsm = lsm.astype(bool)
# mask_arr = lsm.data.astype(float)
# mask_arr[mask_arr==False]=np.nan
# data_arr = dataset_raw[namevar].data
# data_arr = data_arr*mask_arr
# dataset_raw[namevar].data = data_arr

# anoms_std = compute_anomalies(dataset_raw,namevar)
# anoms_std.to_netcdf(f'{path_outputs_anoms}anoms_{name_experiment}.nc')

In [9]:
def process_anoms_and_save(id_experiment):
    name_experiment = unique_names_experiments[id_experiment]
    print(name_experiment)
    where_files = np.where(names_experiments_all==name_experiment)[0]
    files_temp = filenames[where_files]
    
    dataset_raw = extractz500_several_files(files_temp)
    land_mask_file='/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/LANDFRAC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.LANDFRAC.185001-185912.nc'
    namevar='TREFHTMX'
    lsm = xr.open_dataset(land_mask_file).LANDFRAC[0]
    lsm = lsm.sel(lat=dataset_raw.lat,lon=dataset_raw.lon)
    lsm = lsm.astype(bool)
    mask_arr = lsm.data.astype(float)
    mask_arr[mask_arr==False]=np.nan
    data_arr = dataset_raw[namevar].data
    data_arr = data_arr*mask_arr
    dataset_raw[namevar].data = data_arr
    
    anoms_std = compute_anomalies(dataset_raw,namevar)
    anoms_std.to_netcdf(f'{path_outputs_anoms}anoms_{name_experiment}.nc')

In [10]:
from multiprocessing import Pool

num_ids = len(unique_names_experiments)
num_processors = 10

# Create a Pool of worker processes
with Pool(processes=num_processors) as pool:
    # Map the function to the range of IDs
    pool.map(process_anoms_and_save, range(num_ids))

LE2-1031.002_smbbLE2-1091.005_smbbLE2-1121.007_cmip6LE2-1061.004_cmip6LE2-1151.008_smbbLE2-1001.001_cmip6LE2-1181.010_cmip6LE2-1231.002_cmip6LE2-1231.008_cmip6LE2-1231.005_cmip6











/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1011.001_smbb
LE2-1041.003_cmip6
LE2-1131.007_smbb
LE2-1231.009_cmip6
LE2-1231.006_cmip6
LE2-1101.006_cmip6
LE2-1071.004_smbb
LE2-1191.010_smbb
LE2-1231.003_cmip6
LE2-1161.009_cmip6


/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1231.007_cmip6
LE2-1021.002_cmip6
LE2-1231.001_cmip6
LE2-1141.008_cmip6
LE2-1081.005_cmip6
LE2-1111.006_smbb
LE2-1231.004_cmip6
LE2-1051.003_smbb
LE2-1231.010_cmip6
LE2-1171.009_smbb


...

/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1301.012_smbb
LE2-1301.015_smbb
LE2-1301.018_smbb


/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1301.013_smbb
LE2-1301.016_smbb
LE2-1301.019_smbb


/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_50342/3053704963.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may